## London House Price Predictive Model
### Imports

In [21]:
import pandas as pd
from datetime import datetime as dt, timezone as tz

In [ ]:
df = pd.read_csv('data/kaggle_london_house_price_data.csv')

In [6]:
df.dtypes

fullAddress                                               object
postcode                                                  object
country                                                   object
outcode                                                   object
latitude                                                 float64
longitude                                                float64
bathrooms                                                float64
bedrooms                                                 float64
floorAreaSqM                                             float64
livingRooms                                              float64
tenure                                                    object
propertyType                                              object
currentEnergyRating                                       object
rentEstimate_lowerPrice                                  float64
rentEstimate_currentPrice                                float64
rentEstimate_upperPrice  

### Data Cleaning

In [5]:
df['sale_date'] = pd.to_datetime(df['saleEstimate_ingestedAt'])

In [ ]:
key_df = df.drop(columns=['history_numericChange', 'history_percentageChange', 'history_date', 'history_price','saleEstimate_valueChange.saleDate','saleEstimate_valueChange.percentageChange','saleEstimate_ingestedAt','saleEstimate_valueChange.numericChange'])

In [23]:
house_df = key_df[key_df['sale_date'] > dt(2015, 1,1).replace(tzinfo=tz.utc)] #Only considering houses with estimated values in the last 10 years

### EDA

In [27]:
house_df.isna().sum()

fullAddress                         0
postcode                            0
country                             0
outcode                             0
latitude                            0
longitude                           0
bathrooms                       77755
bedrooms                        40404
floorAreaSqM                    25066
livingRooms                     60341
tenure                          11494
propertyType                     1126
currentEnergyRating             84288
rentEstimate_lowerPrice          1101
rentEstimate_currentPrice        1101
rentEstimate_upperPrice          1101
saleEstimate_lowerPrice             0
saleEstimate_currentPrice           0
saleEstimate_upperPrice             0
saleEstimate_confidenceLevel        0
sale_date                           0
dtype: int64

In [ ]:
temp_df = house_df.dropna(subset=['bathrooms', 'livingRooms','bedrooms','floorAreaSqM'])
temp_df.isna().sum() # Cannot impute the number of rooms and area of a building

fullAddress                         0
postcode                            0
country                             0
outcode                             0
latitude                            0
longitude                           0
bathrooms                           0
bedrooms                            0
floorAreaSqM                        0
livingRooms                         0
tenure                           1702
propertyType                        0
currentEnergyRating             51266
rentEstimate_lowerPrice            16
rentEstimate_currentPrice          16
rentEstimate_upperPrice            16
saleEstimate_lowerPrice             0
saleEstimate_currentPrice           0
saleEstimate_upperPrice             0
saleEstimate_confidenceLevel        0
sale_date                           0
dtype: int64

In [ ]:
flat_df = temp_df[temp_df['propertyType'].str.lower().str.contains('flat')]
flat_df['tenure'].value_counts() # 98% of flats are leasehold. Would be therefore valid to impute that any flats w/o tenure are in fact leasehold

tenure
Leasehold    178804
Feudal         2796
Freehold       2039
Shared          451
Name: count, dtype: int64

In [ ]:
flat_df['currentEnergyRating'].value_counts() #Normally distributed around C/D could impute with preference for B,C,D,E for flats

currentEnergyRating
C    69632
D    49279
B    22602
E     9409
F      586
A      105
G      101
Name: count, dtype: int64

In [39]:
non_flat_df = temp_df[~temp_df['propertyType'].str.lower().str.contains('flat')]

In [ ]:
non_flat_df['tenure'].value_counts() #Houses tend to be freehold with 97% of houses Freehold, impute freehold for non flat properties

tenure
Freehold     128580
Leasehold      4128
Feudal           56
Shared           22
Name: count, dtype: int64

In [ ]:
non_flat_df['currentEnergyRating'].value_counts() #Sharper normal distribution centered around D, could impute with preference for C,D,E

currentEnergyRating
D    64439
C    32420
E    15526
B     1714
F     1061
G      329
A      109
Name: count, dtype: int64

This exploration of the data has clear reasoning for imputing some fields and removing rows with missing key fields.
- Bathrooms : Cannot be reliably imputed and is necessary for the future model
- Living Rooms: Cannot be reliably imputed and is necessary for the future model
- Floor Area : Cannot be reliably imputed and is necessary for the future model
- Tenure: Can be reliably imputed and is necessary for the future model
- Energy rating: Can be reliably imputed and is necessary for the future model

### Feature Investigation
#### Outcode vs Postcode